In [66]:
# Import the required libraries and dependencies
import FundamentalAnalysis as fa
import os
import pandas as pd
import hvplot.pandas
from dotenv import load_dotenv
from pathlib import Path

In [59]:
# Load the .env file into the notebook
load_dotenv()

# Read in your FMPCLOUD.IO_API_KEY
api_key = os.getenv("FMPCLOUD_API_KEY")

# Confirm the availability of your FMPCLOUD.IO API access key by checking its type
if not api_key:
    print("Error with the FMPCLOUD.IO API key. Check the .env file.")

In [67]:
# Read in the CSV file called "tickers.csv" using the Path module. 
# The CSV file is located in the Resources folder.
tickers = pd.read_csv(
    Path("./Resources/tickers.csv"),
)

In [68]:
# Use the head (and/or tail) function to confirm that the data was imported properly.
display(tickers.head())

,Ticker,Company
0,AAPL,Apple Inc
1,GOOG,Alphabet Inc
2,FB,"Facebook, Inc"
3,SNOW,Snowflake Inc
4,NVDA,NVIDIA Corporation


In [69]:
# Converting column data to list
tickers = tickers['Ticker'].tolist()
tickers

['AAPL', 'GOOG', 'FB', 'SNOW', 'NVDA']

In [70]:
# Get Return on Equity for all the companies in the dataset
roe_df = pd.DataFrame(columns=['Ticker',
                               'Gross Margin',
                               'Operating Margin',
                               'Net Margin',
                               'Gross Debt/Equity',
                               'Asset Turnover', 
                               'ROE'])

for ticker in tickers:
    balance_sheet_quarterly = fa.balance_sheet_statement(ticker, api_key, period="quarter")
    income_statement_quarterly= fa.income_statement(ticker, api_key, period="quarter")
    cash_flow_statement_quarterly = fa.cash_flow_statement(ticker, api_key, period="quarter")
    key_metrics_quarterly = fa.key_metrics(ticker, api_key, period="quarter")
    financial_ratios_quarterly = fa.financial_ratios(ticker, api_key, period="quarter")
    growth_quarterly = fa.financial_statement_growth(ticker, api_key, period="quarter")
    
    gross_margin=round(financial_ratios_quarterly.loc['grossProfitMargin',financial_ratios_quarterly.columns[0]],3)
    # ebitda_quarterly=round(income_statement_quarterly.loc['ebitda',income_statement_quarterly.columns[0]],3)
    # ebitda_margin = round(ebitda_quarterly/revenue_quarterly,3)
    operating_margin=round(financial_ratios_quarterly.loc['operatingProfitMargin',financial_ratios_quarterly.columns[0]],3)
    net_margin=round(financial_ratios_quarterly.loc['netProfitMargin',financial_ratios_quarterly.columns[0]],3)
    gross_debt_equity = round(key_metrics_quarterly.loc['debtToEquity',key_metrics_quarterly.columns[0]],3)
    revenue_quarterly=round(income_statement_quarterly.loc['revenue',income_statement_quarterly.columns[0]],3)
    assets=round(balance_sheet_quarterly.loc['totalAssets',key_metrics_quarterly.columns[0]],3)
    asset_turnover=round(revenue_quarterly*4/assets,3)
    equity=round(balance_sheet_quarterly.loc['totalStockholdersEquity',balance_sheet_quarterly.columns[0]],3)
    net_profit_quarterly=round(income_statement_quarterly.loc['netIncome',income_statement_quarterly.columns[0]],3)
    roe=round(net_profit_quarterly*4/equity,3)
    roe_module = [ticker,gross_margin,operating_margin,net_margin,gross_debt_equity,asset_turnover,roe]
    roe_df.loc[len(roe_df.index)] = roe_module

roe_df=roe_df.set_index('Ticker')
roe_df

,Gross Margin,Operating Margin,Net Margin,Gross Debt/Equity,Asset Turnover,ROE
Ticker,,,,,,
AAPL,0.425,0.307,0.264,1.570,1.063,1.366
GOOG,0.564,0.297,0.324,0.060,0.676,0.312
FB,0.804,0.435,0.363,0.079,0.640,0.284
SNOW,0.565,-1.052,-1.044,0.000,0.129,-0.161
NVDA,0.631,0.301,0.291,0.353,0.695,0.345


In [65]:
# Create a line plot for the Return on Investment (ROE) DataFrame for the all the companies in the dataset 
roe_df.T.hvplot.line(
    title="ROE Comparison",
    rot=90).opts(yformatter='%.1f')

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [64]:
# Create a bar plot for the Return on Investment (ROE) DataFrame for the all the companies in the dataset 
roe_df.T.hvplot.bar(
    title="ROE Comparison",
    rot=90).opts(yformatter='%.1f')

:Bars   [index,Variable]   (value)

In [ ]:
# Get Forward Cash Flow for all the companies in the dataset
fcf_df = pd.DataFrame(columns=['Ticker','CFWC/CFFO','Capex/CFFO','Capex/D&A','Acquisiton/FCF','FCF/Revs', 'FCF/Equity'])